In [13]:
import pandas as pd
import numpy as np
from sklearn.utils import murmurhash3_32

In [5]:
def cleanNull(DF):
    for column in DF.columns.values:
        replace = DF[column].value_counts()
        index_min = np.argmin(replace)
        DF[column].fillna(index_min , inplace=True)

In [17]:
def generateFeatureEngine(DF , toGenerate):
    for g in toGenerate :
        for i in toGenerate :
            if i != g :
                newName = str(g) + ":" + str(i)
                DF[newName] = newName+DF[i].astype('str')+":"+DF[g].astype('str')
                print newName
                for j in toGenerate :
                    if j!= i and j!=g :
                        newX = newName + ":" + str(j)
                        DF[newName] = newName+DF[newName]+":"+DF[j].astype('str')
                        print newX

In [ ]:
DF = pd.read_csv("4train.txt", header=-1, sep='\t')
DF = pd.DataFrame(DF)
num_Cols=[]
cat_Cols = []
dd = {}
dd[0]="Label"
for i in range(1,14):
    dd[i]="I"+`i`
    num_Cols.append("I"+`i`)
for c in range(1,27):
    dd[c+13]="C"+`c`
    cat_Cols.append("C"+`c`)
DF.rename(columns=dd, inplace=True)
cleanNull(DF)

In [21]:
TOGEN = ['C10', 'C7' , 'C11']
generateFeatureEngine(DF,TOGEN)

C10:C7
C10:C7:C10
C10:C7:C11
C10:C11
C10:C11:C10
C10:C11:C7
C7:C10
C7:C10:C7
C7:C10:C11
C7:C11
C7:C11:C10
C7:C11:C7
C11:C10
C11:C10:C7
C11:C10:C11
C11:C7
C11:C7:C10
C11:C7:C11


In [34]:
DF.to_csv("FinalCri.csv" , header = False , index= False)

In [37]:
Li = ['C10:C7' ,  'C10:C11' ,  'C7:C10' , 'C7:C11','C11:C10' ,'C11:C7','C11:C7:C10']

In [54]:
DF['C11:C7']

0          3.352366e+36
1          6.543753e+31
2          6.691298e+35
3          1.937738e+36
4          1.048944e+35
5          6.522322e+36
6          5.498504e+36
7          2.846101e+35
8          4.119412e+35
9          6.776150e+36
10         4.319599e+35
11         3.194021e+35
12         3.634647e+36
13         6.613090e+36
14         3.785193e+32
15         1.580056e+36
16         5.687776e+35
17         1.121520e+36
18         9.484392e+36
19         3.958188e+36
20         1.054987e+37
21         1.911157e+36
22         1.663430e+36
23         1.445988e+36
24         2.782370e+36
25         7.905569e+36
26         2.274124e+35
27         4.666368e+36
28         2.531238e+35
29         4.930420e+36
               ...     
1999970    8.155001e+34
1999971    9.228981e+36
1999972    2.302909e+36
1999973    8.928122e+35
1999974    1.589065e+33
1999975    4.571373e+36
1999976    4.727051e+36
1999977    6.716049e+36
1999978    1.224976e+36
1999979    3.971669e+34
1999980    6.265

In [38]:
from datetime import datetime
from csv import DictReader
from math import exp, log, sqrt
from sklearn.utils import murmurhash3_32

In [39]:
train = "FinalCri.csv"
header = ["Label"]
header = header+num_Cols+cat_Cols+Li

In [46]:
def logloss(p, y):
    p = max(min(p, 1. - 10e-17), 10e-17)        # The bounds
    return -log(p) if y == 1. else -log(1. - p)
def get_x(csv_row, D):
    fullind = []
    x = {}
    num = {}
    for key, value in csv_row.items():
        if key in cat_Cols:
            s = key + ' ' + value
            fullind.append(murmurhash3_32(s, positive=True) % D)
        else:
            num[key]=value
        
    
    x[0] = 1  # 0 is the index of the bias term
    for index in fullind:
        if(not x.has_key(index)):
            x[index] = 0
        x[index] += 1
    
    return x, num  # x contains indices of features that have a value as number of occurences

def get_p(x, num, w, D):
    wTx = 0.
    for i, xi in x.items():
        wTx += w[i] * xi  # w[i] * x[i]
    for numj in num.values():
        
        wTx += w[murmurhash3_32(''+numj, positive=True) % D] * float(numj)
        
        
    return 1. / (1. + exp(-max(min(wTx, 50.), -50.)))  # bounded sigmoid

def update_w(w, g, x, num, D, p, y):
    for i, xi in x.items():
        g[i] += 1
        w[i] -= (p - y) * xi * alpha / (sqrt(g[i]))  # Minimising log loss
    for numj in num.values():
        numi = murmurhash3_32(''+numj, positive=True) % D
        g[numi] += 1
        w[numi] -= (p - y) * float(numj) * alpha / (sqrt(g[numi]))  # Minimising log loss
    return w, g

In [47]:
logbatch = 100000

D = 2 ** 4    # number of weights use for learning
alpha = .004  # learning rate for sgd optimization

w = [0.] * D  # weights
g = [.5] * D  # sum of historical gradients

# start training a logistic regression model using on pass sgd
loss = 0.
lossb = 0.

for t, row in enumerate(DictReader(open(train),header, delimiter=',')):
    y = 1. if row['Label'] == '1' else 0.

    del row['Label']  # can't let the model peek the answer

    # main training procedure
    # step 1, get the hashed features
    print row
    x, num = get_x(row, D)
    # step 2, get prediction
    p = get_p(x, num, w, D)

    # for progress validation, useless for learning our model
    lossx = logloss(p, y)
    loss += lossx
    lossb += lossx
    if t % logbatch == 0 and t > 1:
        print('%s\t Line : %d\t logloss: %f\t batch logloss: %f' % (datetime.now(), t, loss/t, lossb/logbatch))
        lossb = 0.

    # step 3, update model with answer
    w, g = update_w(w, g, x, num, D, p, y)


{'C10:C7:C11': 'C7:C10', 'C11:C10:C7': None, 'C7:C10:C7': None, 'C7:C11:C7': None, 'C19': 'C19', 'C18': 'C18', 'C13': 'C13', 'C12': 'C12', 'C11': 'C11', 'C10': 'C10', 'C17': 'C17', 'C16': 'C16', 'C15': 'C15', 'C14': 'C14', 'I9': 'I9', 'I8': 'I8', 'I1': 'I1', 'I3': 'I3', 'I2': 'I2', 'I5': 'I5', 'I4': 'I4', 'I7': 'I7', 'I6': 'I6', 'C9': 'C9', 'C8': 'C8', 'C10:C11:C10': 'C11:C10', 'C3': 'C3', 'C2': 'C2', 'C1': 'C1', 'C7:C11': None, 'C7': 'C7', 'C6': 'C6', 'C5': 'C5', 'C4': 'C4', 'C22': 'C22', 'C23': 'C23', 'C20': 'C20', 'C21': 'C21', 'C26': 'C26', 'C24': 'C24', 'C25': 'C25', 'I11': 'I11', 'I10': 'I10', 'I13': 'I13', 'I12': 'I12', 'C7:C10:C11': None, 'C10:C7': 'C10:C7', 'C11:C10:C11': None, 'C10:C11': 'C7:C11', 'C10:C7:C10': 'C10:C11', 'C11:C7': None, 'C11:C7:C11': None, 'C11:C7:C10': None, 'C11:C10': None, 'C7:C11:C10': None, 'C7:C10': None, 'C10:C11:C7': 'C11:C7'}


TypeError: cannot concatenate 'str' and 'NoneType' objects